In [11]:
import pickle 
import pandas as pd
import numpy as np

In [12]:
with open('preprocessed_data/preprocessed_general.pkl', 'rb') as f:
    chosen_stocks = pickle.load(f)

In [13]:
chosen_stocks[2][13].shape

(1226, 64)

In [14]:
def create_lag_values(data, columns, lag):
    df = pd.DataFrame(index=data.index)
    for i in range(1, lag + 1):
        df[[f"{col}_lag_{i}" for col in columns]] = data[columns].shift(i)
    return df

lags_5 = ['close', 'open', 'high', 'low', 'volume', 'vwap']
lags_3 = ['rsi', 'roc_9', 'roc_14', 'roc_21', 'macd', 'macd_signal', 'macd_hist',
          'stoch_%K', 'stoch_%D', 'stoch_diff', 'cci', 'plus_di', 'minus_di', 'dx', 'adx',
          'obv', 'obv_norm', 'obv_momentum', 'vpt', 'vpt_norm', 'vpt_momentum']

# Create lists to hold the data all bins
preprocessed_Tree_data = []
for i in range(len(chosen_stocks)):
    preprocessed_Tree_bin = []
    for j in range(len(chosen_stocks[i])):
        chosen_stocks[i][j].drop(columns=['symbol'], inplace=True)
        lags_5_df = create_lag_values(chosen_stocks[i][j], lags_5, 5)
        lags_3_df = create_lag_values(chosen_stocks[i][j], lags_3, 3)
        chosen_stocks[i][j] = pd.concat([chosen_stocks[i][j], lags_5_df, lags_3_df], axis=1)
        chosen_stocks[i][j].dropna(inplace=True)

        n = len(chosen_stocks[i][j])
        end_train = int(n * 0.8)
        end_val = int(n * 0.9)

        train_df = chosen_stocks[i][j].iloc[:end_train]
        val_df = chosen_stocks[i][j].iloc[end_train:end_val]
        test_df = chosen_stocks[i][j].iloc[end_val:]

        X_train = train_df.drop(columns=['target', 'target_ret', 'target_ret_log'])
        y_train = train_df['target']
        X_validation = val_df.drop(columns=['target', 'target_ret', 'target_ret_log'])
        y_validation = val_df['target']
        X_test = test_df.drop(columns=['target', 'target_ret', 'target_ret_log'])
        y_test = test_df['target']

        entry = (X_train, y_train, X_validation, y_validation, X_test, y_test)
        preprocessed_Tree_bin.append(entry)
    preprocessed_Tree_data.append(preprocessed_Tree_bin)     

with open(f'preprocessed_data/preprocessed_XGBoost.pkl', 'wb') as f:
    pickle.dump(preprocessed_Tree_data, f)